In [5]:
#%pip install pandas
#%pip install openpyxl

In [1]:
import pandas as pd
import openpyxl
import networkx as nx

In [18]:
# Definiciones globales

%run ../Lib/pandas_options.py
%run ../Lib/define_constantes.py

In [20]:
#leer archivo plano del "DataLake"

archivo_datos_tramos = f'{DIRECTORIO_DATOS_CRUDOS}/DATOS_TRAMOS_GRAFO.csv'
tuberias_df = pd.read_csv(archivo_datos_tramos, low_memory=False)

In [21]:
tuberias_df.shape

(178027, 5)

In [22]:
tuberias_df.head()

,G3E_FID,NODO1_ID,NODO2_ID,LONGITUD,ARRANQUE
0,10319871,9566618,9416102,50.27,NO
1,10205159,9504506,9505087,4.25,NO
2,310813,6054051,6054073,11.11,NO
3,311445,6054344,6054345,44.40,NO
4,311452,6054350,6054351,34.20,SI


In [23]:
# Crear un grafo dirigido
G = nx.DiGraph()

In [24]:
# Agregar todas las tuberías al grafo
for _, row in tuberias_df.iterrows():
    G.add_edge(
        row['NODO1_ID'],
        row['NODO2_ID'],
        G3E_FID=row['G3E_FID'],
        length=row['LONGITUD'],
        arranque=row['ARRANQUE']
    )


In [25]:
# Función para calcular la longitud de las redes aguas arriba para un nodo específico
def calcular_longitud_aguas_arriba(nodo, grafo):
    # Lista para almacenar los nodos visitados y la longitud total
    visitados = set()
    longitud_total = 0

    # Función recursiva para recorrer los nodos aguas arriba
    def recorrer_aguas_arriba(nodo_actual):
        nonlocal longitud_total
        visitados.add(nodo_actual)

        # Recorrer todos los predecesores (nodos aguas arriba)
        for predecesor in grafo.predecessors(nodo_actual):
            if predecesor not in visitados:
                # Sumar la longitud de la tubería que conecta el predecesor con el nodo actual
                longitud_total += grafo[predecesor][nodo_actual]['length']

                # Verificar si la arista tiene arranque = 'SI' y detener el recorrido en esa ruta
                if grafo[predecesor][nodo_actual]['arranque'] == 'SI':
                    continue  # Detener el recorrido por este camino, pero ya se sumó la longitud

                # Recursión para seguir buscando más arriba
                recorrer_aguas_arriba(predecesor)

    # Iniciar el recorrido desde el nodo especificado
    recorrer_aguas_arriba(nodo)

    return longitud_total


In [26]:
# Crear una nueva columna en el DataFrame para almacenar la longitud aguas arriba de cada tubería
tuberias_df['LONGITUD_AGUAS_ARRIBA'] = tuberias_df['NODO1_ID'].apply(lambda nodo: calcular_longitud_aguas_arriba(nodo, G))

In [27]:
#Asignar longitud_aguas_arriba = 0 para los arranques.
tuberias_df.loc[tuberias_df['ARRANQUE'] == 'SI', 'LONGITUD_AGUAS_ARRIBA'] = 0

In [12]:
tuberias_df.head()

,G3E_FID,NODO1_ID,NODO2_ID,LONGITUD,ESTADO,CUENCA,ARRANQUE,LONGITUD_AGUAS_ARRIBA
0,9223134,9113886,6085537,9.34,OPERACION,LA HUESO,NO,7837.00
1,9224406,9114334,9114335,15.04,OPERACION,LA MALPASO,NO,9.10
2,9224407,9114335,9114336,3.06,OPERACION,LA MALPASO,NO,24.14
3,9224409,9114337,9114338,13.12,OPERACION,LA MALPASO,NO,34.65
4,9224410,9114338,9114339,3.61,OPERACION,LA MALPASO,NO,47.77


In [29]:
# Ruta donde deseas guardar el archivo
archivo_datos_long_aguas_arr = f'{DIRECTORIO_DATOS_CRUDOS}/DATOS_LONGITUD_AGUAS_ARRIBA.csv'

In [30]:

# Guardar el DataFrame como un archivo Excel
tuberias_df.to_csv(archivo_datos_long_aguas_arr, index=False)